In [1]:
from tqdm import tqdm
import pandas as pd
import re
import emoji
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModel
from soynlp.normalizer import repeat_normalize
import numpy as np

c:\python-capstone\review_tag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 1: 데이터 로드
file_path = "C:/Users/cho03/Desktop/대학/4학년/2학기/캡스톤디자인-여행사이트/data/lodging_reviews.csv"
data = pd.read_csv(file_path)

# Step 2: 'review_text' 또는 'rating' 컬럼에 NaN이 있는 행 삭제
data = data.dropna(subset=["review_text", "rating"])

In [3]:
# 전처리 함수 정의
pattern = re.compile(f"[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+")
url_pattern = re.compile(
    r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"
)

def clean(x):
    x = pattern.sub(" ", x)
    x = emoji.replace_emoji(x, replace="")  # emoji 삭제
    x = url_pattern.sub("", x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

# Step 3: 리뷰 텍스트 전처리
data["review_text"] = data["review_text"].apply(clean)

In [4]:
# Step 4: 리뷰의 길이를 계산하여 'length' 컬럼에 저장
data["length"] = data["review_text"].str.len()

# Step 5: 리뷰의 길이가 10 미만인 데이터 제거
data = data[data["length"] >= 10]

# Step 6: 각 숙소(lodging_id)별 전체 리뷰를 기준으로 평균 평점과 리뷰 개수 계산
lodging_summary = (
    data.groupby("lodging_id")
    .agg(total_review_count=("rating", "count"), average_rating=("rating", "mean"))
    .reset_index()
)

In [5]:
# 조건 1: 전체 리뷰의 평균 평점이 4 이상인 숙소
qualified_lodging_ids = lodging_summary[lodging_summary["average_rating"] >= 4]["lodging_id"]

# Step 7: 조건을 만족하는 숙소 중 평점이 4 이상인 리뷰만 필터링
good_review = data[(data["lodging_id"].isin(qualified_lodging_ids)) & (data["rating"] >= 4)]

# Step 8: 평점이 4 이상인 리뷰 개수가 5개 이상인 숙소만 선택
final_lodging_summary = good_review.groupby("lodging_id").agg(good_review_count=("rating", "count")).reset_index()
final_lodging_ids = final_lodging_summary[final_lodging_summary["good_review_count"] >= 5]["lodging_id"]

# Step 9: 최종 필터링 - 전체 조건을 만족하는 숙소의 평점 4 이상인 리뷰만 추출
filtered_data = good_review[good_review["lodging_id"].isin(final_lodging_ids)]

In [6]:
filtered_data

,lodging_id,rating,review_text,photos,date,length
27,6,5,깨끗하고 깔끔합니다. 제주일주 하면서 이렇게 깨끗한곳은 처음이네요. 방충망도 꼼꼼하...,NaN,2024.10.25.,177
29,6,5,제가 게스트하우스를 많이 가보지는 않았지만 정말 최고의 숙소라고 할 수 있습니다. ...,NaN,2023.02.17.,158
30,6,4,뷰가 일단 대박이구요 룸 컨디션은 쏘쏘했는데 세탁실 쓸수있고 공용 라운지에서 조식먹...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.10.01.,71
31,6,5,제주에 오면 꼭 다시 오고 싶은 곳.넓고 깔끔 쾌적.세탁기실도 갖춰 편리.편백나무 ...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.07.02.,189
32,6,5,사장님의 세심함이 느껴지는 게하. 서울부터 전국여행 중인 부부입니다. 수 많은 게하...,NaN,2022.06.26.,386
...,...,...,...,...,...,...
46559,3822,5,아주 작지만 있을건 다있는 정갈한곳이었다 아침조식은 12첩반상정도로 깔끔한 밥상이었...,NaN,2024.04.07.,78
46561,3822,5,찐짜 청결함 방은 작지만 바닥 따뜻하고 물 온도도 조절 잘됨,NaN,2023.03.26.,33
46563,3822,5,덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...,NaN,2023.01.03.,311
46564,3822,5,새건물처럼 시설청결하고 방이 무척 따끈해요. 침대도 커서 편해요. 특히 조식 기대이...,NaN,2022.01.14.,68


In [7]:
# 태그별 예시 문장 정의
tag_examples = {
    "가성비": [
        "이 가격에 이 정도 퀄리티면 정말 만족스럽습니다.",
        "가성비로는 더할 나위 없이 좋았습니다.",
        "비슷한 가격대의 다른 숙소보다 훨씬 나았어요.",
        "경제적인 가격에 품질이 뛰어났습니다.",
        "저렴한데도 불구하고 필요한 것이 다 갖춰져 있었어요.",
        "가격 대비 너무 잘 갖춰져 있어 만족스러웠습니다.",
        "가격에 비해 방이 훨씬 넓고 쾌적했어요.",
        "이 정도 시설에 이 가격이라니, 정말 알뜰하게 다녀갑니다.",
        "적은 비용으로 만족스러운 경험을 했어요.",
        "부담 없는 가격으로 편안하게 묵을 수 있었습니다.",
    ],
    "청결": [
        "객실은 물론이고, 구석구석이 반짝반짝하게 청소되어 있었어요.",
        "침구에서 깔끔하고 깨끗한 느낌이 들었습니다.",
        "화장실이 깨끗하고 정리정돈이 잘 되어 있어서 좋았습니다.",
        "바닥부터 천장까지 먼지 한 톨 없이 관리가 잘 되었어요.",
        "입구부터 청결함이 느껴지는 숙소였습니다.",
        "모든 시설이 청결하게 관리되고 있어서 안심이 되었어요.",
        "손 닿는 곳마다 위생적이라 기분 좋게 이용할 수 있었어요.",
        "위생이 철저하게 관리되고 있어 편안하게 머물렀습니다.",
        "전체적으로 정돈이 잘 되어 있어 신경을 많이 쓴 느낌이었어요.",
        "어디 하나 더럽거나 정리 안 된 곳이 없어서 만족스러웠습니다.",
    ],
    "직원 만족": [
        "직원분들이 친절하고 미소로 맞아주셔서 편안했습니다.",
        "모든 요청에 빠르게 응대해주셔서 감사했습니다.",
        "체크인부터 체크아웃까지 직원들이 세심하게 신경 써주셨어요.",
        "직원분들이 하나하나 고객의 편의를 생각해주는 느낌이었어요.",
        "문의할 때마다 친절하게 답변해 주셔서 좋았습니다.",
        "직원들이 항상 인사하며 고객을 배려하는 모습이 보기 좋았습니다.",
        "필요한 요청에 빠르게 반응해주셔서 도움이 많이 됐습니다.",
        "서비스가 아주 좋고, 친절함이 돋보이는 곳이었습니다.",
        "직원들이 항상 상냥하고 긍정적인 태도를 보여 주었습니다.",
        "정말 사소한 요청에도 성심껏 응대해주셔서 감사했습니다.",
    ],
    "위치": [
        "관광 명소와 가까워 이동하기에 정말 편리했습니다.",
        "걸어서 갈 수 있는 거리에 다양한 편의시설이 있어 좋았어요.",
        "주요 상권과 가까워서 식사나 쇼핑도 편리하게 할 수 있었어요.",
        "대중교통이 잘 갖춰진 지역이라 이동이 쉬웠습니다.",
        "숙소가 주요 도로와 인접해 있어 차량 접근성이 좋았습니다.",
        "조용한 동네에 위치해 있어 휴식하기 딱 좋은 곳이에요.",
        "관광지와 가까워 여행 계획이 수월했습니다.",
        "숙소가 주요 장소와 가까워 이동할 때 매우 편리했어요.",
        "관광을 위한 최적의 위치, 어디든 쉽게 갈 수 있었습니다.",
        "숙소 위치 덕분에 일정 소화가 더 쉬워서 좋았어요.",
    ],
    "가족 여행": [
        "가족 단위로 방문하기에 안전하고 편리한 숙소입니다.",
        "아이들이 즐길 수 있는 공간이 마련되어 있어 좋았습니다.",
        "숙소에서 아이들과 함께 다양한 액티비티를 즐길 수 있었어요.",
        "가족 모두 편안하게 지낼 수 있는 숙소였어요.",
        "아이와 함께 여행하기에 최적화된 시설이 갖춰져 있었습니다.",
        "주변에 가족이 함께 즐길 수 있는 볼거리와 놀거리가 많아요.",
        "어린 자녀와 함께하기에 안성맞춤인 숙소였어요.",
        "편안한 분위기 덕분에 가족 모두 즐겁게 머물렀습니다.",
        "아이들까지 배려한 세심한 시설이 인상적이었습니다.",
        "가족 단위로 편안하게 쉴 수 있도록 잘 갖춰져 있었습니다.",
    ],
    "연인": [
        "로맨틱한 분위기로 연인과 함께하기에 정말 좋았어요.",
        "둘만의 시간을 보내기에 아늑하고 따뜻한 숙소였습니다.",
        "연인과 기념일에 방문하기에 안성맞춤이었어요.",
        "특별한 날, 특별한 분위기 덕분에 추억이 될 만한 장소였습니다.",
        "연인과 함께 머물기 좋은 프라이빗한 공간이었습니다.",
        "숙소 분위기가 로맨틱해서 커플들에게 추천합니다.",
        "특별한 하루를 보낼 수 있는 연인 추천 숙소입니다.",
        "연인과 둘만의 시간을 즐기기 완벽한 장소였어요.",
        "둘만의 로맨틱한 시간을 보내기에 너무나도 좋았습니다.",
        "연인과 함께라면 더 특별하게 느껴지는 숙소였어요.",
    ],
    "풍경": [
        "창 밖으로 펼쳐진 풍경이 정말 인상적이었어요.",
        "숙소에서 멋진 산과 호수를 바라볼 수 있어 좋았습니다.",
        "숙소가 주변 자연 경관과 잘 어우러져 힐링이 되었습니다.",
        "탁 트인 전망이 일상 스트레스를 날려주는 느낌이었어요.",
        "바다와 하늘이 맞닿은 아름다운 풍경을 즐길 수 있었습니다.",
        "눈 앞에 펼쳐진 멋진 풍경 덕분에 여행의 질이 높아졌어요.",
        "자연 속에 있는 듯한 경치 덕분에 기분이 편안해졌습니다.",
        "일출과 일몰을 감상하기에 최적의 위치였어요.",
        "환상적인 뷰 덕분에 사진 찍기 좋은 장소였습니다.",
        "자연을 그대로 느낄 수 있는 아름다운 숙소였습니다.",
    ],
}

In [ ]:
# KCBERT 모델 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
model = AutoModel.from_pretrained("beomi/kcbert-base")


# 문장 임베딩 함수 정의
def get_kcbert_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    return embedding  # 1차원 배열로 반환


# 태그 예시 문장 임베딩 생성 (풀어서 작성) with tqdm
tag_embeddings = {}
for tag, sentences in tqdm(tag_examples.items(), desc="예시 문장 임베딩 중"):
    embeddings = [get_kcbert_embedding(sentence) for sentence in sentences]
    tag_embeddings[tag] = embeddings

예시 문장 임베딩 중:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# 리뷰 문장 임베딩 생성
review_embeddings = []
for review_text in tqdm(filtered_data["review_text"], desc="리뷰 문장 임베딩 중"):
    embedding = get_kcbert_embedding(review_text)
    review_embeddings.append(embedding)

리뷰 문장 임베딩 중:  16%|█▋        | 1446/8789 [04:17<24:56,  4.91it/s]  

In [ ]:
# 유사도 임계값 설정
threshold = 0.495
assigned_tags_per_review = []
for review_embedding in tqdm(review_embeddings, desc="리뷰별 태그 할당 중"):
    tag_scores = {}
    for tag, embeddings in tag_embeddings.items():
        similarity = cosine_similarity([review_embedding], embeddings).mean()
        tag_scores[tag] = similarity
    review_tags = [tag for tag, score in tag_scores.items() if score > threshold]
    assigned_tags_per_review.append(review_tags)

filtered_data["assigned_tags"] = assigned_tags_per_review

리뷰별 태그 할당 중:   0%|          | 0/8789 [00:00<?, ?it/s]

리뷰별 태그 할당 중: 100%|██████████| 8789/8789 [00:46<00:00, 188.74it/s]
C:\Users\cho03\AppData\Local\Temp\ipykernel_20304\2070289260.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["assigned_tags"] = assigned_tags_per_review


In [ ]:
filtered_data

,lodging_id,rating,review_text,photos,date,length,assigned_tags
27,6,5,깨끗하고 깔끔합니다 제주일주 하면서 이렇게 깨끗한곳은 처음이네요 방충망도 꼼꼼하게 ...,NaN,2024.10.25.,177,"[청결, 위치, 가족 여행, 풍경]"
29,6,5,제가 게스트하우스를 많이 가보지는 않았지만 정말 최고의 숙소라고 할 수 있습니다 시...,NaN,2023.02.17.,161,"[가성비, 청결, 직원 만족, 위치, 가족 여행, 연인, 풍경]"
30,6,4,뷰가 일단 대박이구요 룸 컨디션은 쏘쏘했는데 세탁실 쓸수있고 공용 라운지에서 조식먹...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.10.01.,71,"[가성비, 청결, 위치, 가족 여행, 연인, 풍경]"
31,6,5,제주에 오면 꼭 다시 오고 싶은 곳넓고 깔끔 쾌적세탁기실도 갖춰 편리편백나무 향나는...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.07.02.,189,"[위치, 가족 여행, 연인, 풍경]"
32,6,5,사장님의 세심함이 느껴지는 게하 서울부터 전국여행 중인 부부입니다 수 많은 게하나 ...,NaN,2022.06.26.,386,"[가성비, 청결, 직원 만족, 위치, 가족 여행, 연인, 풍경]"
...,...,...,...,...,...,...,...
46559,3822,5,아주 작지만 있을건 다있는 정갈한곳이었다 아침조식은 12첩반상정도로 깔끔한 밥상이었...,NaN,2024.04.07.,78,"[가성비, 청결, 위치, 가족 여행, 연인, 풍경]"
46561,3822,5,찐짜 청결함 방은 작지만 바닥 따뜻하고 물 온도도 조절 잘됨,NaN,2023.03.26.,37,"[청결, 위치]"
46563,3822,5,덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...,NaN,2023.01.03.,311,"[청결, 연인, 풍경]"
46564,3822,5,새건물처럼 시설청결하고 방이 무척 따끈해요 침대도 커서 편해요 특히 조식 기대이상입...,NaN,2022.01.14.,68,"[청결, 직원 만족, 위치, 가족 여행, 풍경]"


In [ ]:
# 리뷰별 태그 빈도 계산
review_tag_counts = []
for tags in tqdm(filtered_data["assigned_tags"], desc="리뷰별 태그 및 빈도 계산"):
    tag_counts = Counter(tags)
    review_tag_counts.append(
        {
            "리뷰의_가성비": tag_counts.get("가성비", 0),
            "리뷰의_청결도": tag_counts.get("청결", 0),
            "리뷰의_직원_만족": tag_counts.get("직원 만족", 0),
            "리뷰의_위치": tag_counts.get("위치", 0),
            "리뷰의_가족_여행": tag_counts.get("가족 여행", 0),
            "리뷰의_연인": tag_counts.get("연인", 0),
            "리뷰의_풍경": tag_counts.get("풍경", 0),
        }
    )

# 리뷰별 태그 빈도 리스트를 데이터프레임으로 변환
review_tag_counts_df = pd.DataFrame(review_tag_counts)

# 리뷰 데이터에 리뷰별 태그 빈도 추가
final_review_data = pd.concat([filtered_data.reset_index(drop=True), review_tag_counts_df], axis=1)

리뷰별 태그 및 빈도 계산:   0%|          | 0/8789 [00:00<?, ?it/s]

리뷰별 태그 및 빈도 계산: 100%|██████████| 8789/8789 [00:00<00:00, 111232.02it/s]


In [ ]:
final_review_data

,lodging_id,rating,review_text,photos,date,length,assigned_tags,리뷰의_가성비,리뷰의_청결도,리뷰의_직원_만족,리뷰의_위치,리뷰의_가족_여행,리뷰의_연인,리뷰의_풍경
0,6,5,깨끗하고 깔끔합니다 제주일주 하면서 이렇게 깨끗한곳은 처음이네요 방충망도 꼼꼼하게 ...,NaN,2024.10.25.,177,"[청결, 위치, 가족 여행, 풍경]",0,1,0,1,1,0,1
1,6,5,제가 게스트하우스를 많이 가보지는 않았지만 정말 최고의 숙소라고 할 수 있습니다 시...,NaN,2023.02.17.,161,"[가성비, 청결, 직원 만족, 위치, 가족 여행, 연인, 풍경]",1,1,1,1,1,1,1
2,6,4,뷰가 일단 대박이구요 룸 컨디션은 쏘쏘했는데 세탁실 쓸수있고 공용 라운지에서 조식먹...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.10.01.,71,"[가성비, 청결, 위치, 가족 여행, 연인, 풍경]",1,1,0,1,1,1,1
3,6,5,제주에 오면 꼭 다시 오고 싶은 곳넓고 깔끔 쾌적세탁기실도 갖춰 편리편백나무 향나는...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.07.02.,189,"[위치, 가족 여행, 연인, 풍경]",0,0,0,1,1,1,1
4,6,5,사장님의 세심함이 느껴지는 게하 서울부터 전국여행 중인 부부입니다 수 많은 게하나 ...,NaN,2022.06.26.,386,"[가성비, 청결, 직원 만족, 위치, 가족 여행, 연인, 풍경]",1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,3822,5,아주 작지만 있을건 다있는 정갈한곳이었다 아침조식은 12첩반상정도로 깔끔한 밥상이었...,NaN,2024.04.07.,78,"[가성비, 청결, 위치, 가족 여행, 연인, 풍경]",1,1,0,1,1,1,1
8785,3822,5,찐짜 청결함 방은 작지만 바닥 따뜻하고 물 온도도 조절 잘됨,NaN,2023.03.26.,37,"[청결, 위치]",0,1,0,1,0,0,0
8786,3822,5,덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...,NaN,2023.01.03.,311,"[청결, 연인, 풍경]",0,1,0,0,0,1,1
8787,3822,5,새건물처럼 시설청결하고 방이 무척 따끈해요 침대도 커서 편해요 특히 조식 기대이상입...,NaN,2022.01.14.,68,"[청결, 직원 만족, 위치, 가족 여행, 풍경]",0,1,1,1,1,0,1


In [ ]:
# 숙소별 태그 빈도 계산 및 상위 2개 태그 선정
final_tags_per_lodging = []
lodging_ids = final_review_data["lodging_id"].unique()

for lodging_id in tqdm(lodging_ids, desc="숙소별 태그 및 빈도 계산"):
    keywords_all_reviews = []
    for tags in final_review_data[final_review_data["lodging_id"] == lodging_id]["assigned_tags"]:
        keywords_all_reviews.extend(tags)
    tag_counts = Counter(keywords_all_reviews)
    top_2_tags = [tag for tag, count in tag_counts.most_common(2) if count >= 3]
    final_tags_per_lodging.append(
        {
            "lodging_id": lodging_id,
            "숙소의_가성비": tag_counts.get("가성비", 0),
            "숙소의_청결도": tag_counts.get("청결", 0),
            "숙소의_직원_만족": tag_counts.get("직원 만족", 0),
            "숙소의_위치": tag_counts.get("위치", 0),
            "숙소의_가족_여행": tag_counts.get("가족 여행", 0),
            "숙소의_연인": tag_counts.get("연인", 0),
            "숙소의_풍경": tag_counts.get("풍경", 0),
            "숙소의_top_2_tags": top_2_tags,
        }
    )

숙소별 태그 및 빈도 계산:   0%|          | 0/421 [00:00<?, ?it/s]

숙소별 태그 및 빈도 계산: 100%|██████████| 421/421 [00:00<00:00, 1354.70it/s]


In [ ]:
lodging_tags_specified_df = pd.DataFrame(final_tags_per_lodging)
final_data = pd.merge(final_review_data, lodging_tags_specified_df, on="lodging_id", how="left")

In [ ]:
final_data.to_csv("review_tags_kcbert(0.495).csv", encoding="utf-8-sig", index=False)

In [ ]:
final_data

,lodging_id,rating,review_text,photos,date,length,assigned_tags,리뷰의_가성비,리뷰의_청결도,리뷰의_직원_만족,...,리뷰의_연인,리뷰의_풍경,숙소의_가성비,숙소의_청결도,숙소의_직원_만족,숙소의_위치,숙소의_가족_여행,숙소의_연인,숙소의_풍경,숙소의_top_2_tags
0,6,5,깨끗하고 깔끔합니다 제주일주 하면서 이렇게 깨끗한곳은 처음이네요 방충망도 꼼꼼하게 ...,NaN,2024.10.25.,177,"[청결, 위치, 가족 여행, 풍경]",0,1,0,...,0,1,4,9,6,9,8,6,8,"[청결, 위치]"
1,6,5,제가 게스트하우스를 많이 가보지는 않았지만 정말 최고의 숙소라고 할 수 있습니다 시...,NaN,2023.02.17.,161,"[가성비, 청결, 직원 만족, 위치, 가족 여행, 연인, 풍경]",1,1,1,...,1,1,4,9,6,9,8,6,8,"[청결, 위치]"
2,6,4,뷰가 일단 대박이구요 룸 컨디션은 쏘쏘했는데 세탁실 쓸수있고 공용 라운지에서 조식먹...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.10.01.,71,"[가성비, 청결, 위치, 가족 여행, 연인, 풍경]",1,1,0,...,1,1,4,9,6,9,8,6,8,"[청결, 위치]"
3,6,5,제주에 오면 꼭 다시 오고 싶은 곳넓고 깔끔 쾌적세탁기실도 갖춰 편리편백나무 향나는...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.07.02.,189,"[위치, 가족 여행, 연인, 풍경]",0,0,0,...,1,1,4,9,6,9,8,6,8,"[청결, 위치]"
4,6,5,사장님의 세심함이 느껴지는 게하 서울부터 전국여행 중인 부부입니다 수 많은 게하나 ...,NaN,2022.06.26.,386,"[가성비, 청결, 직원 만족, 위치, 가족 여행, 연인, 풍경]",1,1,1,...,1,1,4,9,6,9,8,6,8,"[청결, 위치]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,3822,5,아주 작지만 있을건 다있는 정갈한곳이었다 아침조식은 12첩반상정도로 깔끔한 밥상이었...,NaN,2024.04.07.,78,"[가성비, 청결, 위치, 가족 여행, 연인, 풍경]",1,1,0,...,1,1,2,5,2,4,3,3,4,"[청결, 위치]"
8785,3822,5,찐짜 청결함 방은 작지만 바닥 따뜻하고 물 온도도 조절 잘됨,NaN,2023.03.26.,37,"[청결, 위치]",0,1,0,...,0,0,2,5,2,4,3,3,4,"[청결, 위치]"
8786,3822,5,덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...,NaN,2023.01.03.,311,"[청결, 연인, 풍경]",0,1,0,...,1,1,2,5,2,4,3,3,4,"[청결, 위치]"
8787,3822,5,새건물처럼 시설청결하고 방이 무척 따끈해요 침대도 커서 편해요 특히 조식 기대이상입...,NaN,2022.01.14.,68,"[청결, 직원 만족, 위치, 가족 여행, 풍경]",0,1,1,...,0,1,2,5,2,4,3,3,4,"[청결, 위치]"


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 각 태그와 리뷰 간의 유사도 평균 계산
average_similarities = []
for i, review_embedding in enumerate(review_embeddings):
    review_tags = assigned_tags_per_review[i]
    tag_similarities = []
    for tag in review_tags:
        tag_embedding = tag_embeddings[tag]
        similarity = cosine_similarity([review_embedding], tag_embedding).mean()
        tag_similarities.append(similarity)
    if tag_similarities:
        average_similarities.append(np.mean(tag_similarities))

# 평균 유사도 확인
print("리뷰-태그 유사도 평균:", np.mean(average_similarities))

리뷰-태그 유사도 평균: 0.54221225
